<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. RSI


> 3. MACD



> 4. Bollinger Bänder












In [ ]:
#indicator = pd.DataFrame(df, index=df.index)

In [ ]:
def calc_indicators(df):
  ema_short = 12
  ema_long = 26
  bollinger_avg = 20
  signalline_span = 9

  df['short_avg_ma'] = df['Close'].rolling(window=bollinger_avg).mean()
  df['short_avg_std'] = df['Close'].rolling(window=bollinger_avg).std()

  df['bollinger'] = df['Close'].ewm(span=bollinger_avg, adjust=False).mean()
  df['bollinger_lower_band'] = df['short_avg_ma'] - (df['short_avg_std'] *2)
  df['bollinger_upper_band'] = df['short_avg_ma'] + (df['short_avg_std'] *2)

  df['ema_short'] = df['Close'].ewm(span=12, adjust=False).mean()
  df['ema_long'] = df['Close'].ewm(span=26, adjust=False).mean()
  df['macd'] = df['ema_short'] - df['ema_long']
  df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

  df = calculate_dead_cat_bounce_v2(df)

  return df

In [ ]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [ ]:
def calc_score(id_list):
  start = dt.datetime(2019,1,1)
  end = dt.datetime.now()
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [ ]:
df_complete = calc_score(['AMZN', 'PLUG', 'AAPL'])
df_complete[df_complete.dead_cat_bounce==True]

,High,Low,Open,Close,Volume,Adj Close,stock_id,short_avg_ma,short_avg_std,bollinger,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,
2020-02-28,1889.760010,1811.130005,1814.630005,1883.750000,9493800.0,1883.750000,AMZN,2063.302502,89.953940,2013.641686,1883.394623,2243.210382,2008.817617,2004.272355,4.545262,40.575703,True
2020-03-12,1765.000000,1675.000000,1721.979980,1676.609985,11346200.0,1676.609985,AMZN,1972.158002,136.463919,1918.576288,1699.230163,2245.085840,1877.943363,1931.180990,-53.237627,-20.869057,True
2019-05-08,2.490000,2.260000,2.300000,2.420000,7494900.0,2.420000,PLUG,2.539000,0.063154,2.500531,2.412692,2.665308,2.501617,2.480769,0.020848,0.048499,True
2019-05-09,2.380000,2.160000,2.360000,2.310000,6230700.0,2.310000,PLUG,2.521000,0.074190,2.482386,2.372619,2.669381,2.472138,2.468120,0.004018,0.039603,True
2019-06-19,2.350000,2.200000,2.310000,2.290000,4944300.0,2.290000,PLUG,2.543500,0.103988,2.503075,2.335525,2.751475,2.482806,2.504171,-0.021365,0.015591,True
2020-03-12,3.490000,3.200000,3.230000,3.270000,20183700.0,3.270000,PLUG,4.565500,0.615600,4.285230,3.334301,5.796699,4.162972,4.297237,-0.134265,0.036294,True
2021-02-18,54.480000,50.060001,51.880001,50.230000,34222100.0,50.230000,PLUG,64.094500,4.694103,60.514456,54.706293,73.482707,60.716823,59.106783,1.610041,4.333278,True
2021-03-25,34.180000,30.100000,31.000000,34.099998,35469800.0,34.099998,PLUG,41.876500,5.337900,41.900997,31.200699,52.552301,39.032677,43.541913,-4.509236,-3.954665,True
2020-02-27,71.500000,68.239998,70.275002,68.379997,320605600.0,67.854019,AAPL,78.426999,3.654900,76.994656,71.117199,85.736800,75.926954,77.103655,-1.176700,0.302785,True


In [ ]:
df_complete

,High,Low,Open,Close,Volume,Adj Close,stock_id,short_avg_ma,short_avg_std,bollinger,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,
2019-01-30,1676.949951,1619.680054,1623.000000,1670.430054,5783800.0,1670.430054,AMZN,1636.097510,50.517984,1631.235727,1535.061542,1737.133477,1643.625283,1621.697891,21.927392,26.556323,False
2019-01-31,1736.410034,1679.079956,1692.849976,1718.729980,10910300.0,1718.729980,AMZN,1645.077509,48.287455,1639.568513,1548.502599,1741.652418,1655.179852,1628.885453,26.294399,26.503938,False
2019-02-01,1673.060059,1622.010010,1638.880005,1626.229980,11506200.0,1626.229980,AMZN,1651.375006,34.715109,1638.298176,1581.944788,1720.805225,1650.726025,1628.688752,22.037274,25.610605,False
2019-02-04,1649.630005,1613.500000,1623.000000,1633.310059,4929100.0,1633.310059,AMZN,1654.271008,30.159684,1637.823118,1593.951640,1714.590376,1648.046646,1629.031071,19.015575,24.291599,False
2019-02-05,1665.260010,1642.500000,1643.339966,1658.810059,4453100.0,1658.810059,AMZN,1655.736011,29.600049,1639.821874,1596.535912,1714.936110,1649.702556,1631.236922,18.465634,23.126406,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-07,127.919998,125.139999,125.830002,127.900002,83466700.0,127.900002,AAPL,122.604000,2.409225,123.563361,117.785550,127.422450,123.764921,123.850159,-0.085239,-1.100551,False
2021-04-08,130.389999,128.520004,128.949997,130.360001,88844600.0,130.360001,AAPL,123.123000,2.885225,124.210660,117.352550,128.893449,124.779548,124.332370,0.447179,-0.791005,False
2021-04-09,133.039993,129.470001,129.800003,133.000000,106513800.0,133.000000,AAPL,123.675000,3.614841,125.047740,116.445318,130.904681,126.044233,124.974416,1.069817,-0.418841,False


In [ ]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

In [ ]:
dead_cat_bounce = calculate_dead_cat_bounce()
#bot_warn_me('Watch Out! Check Code')
dead_cat_bounce